In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from minorminer import find_embedding
from dwave.system import DWaveSampler, EmbeddingComposite

# D-Wave API設定
token = 'your-token'
endpoint = 'https://cloud.dwavesys.com/sapi/'

# プレイヤー数と選択肢の設定
choices = [3,4,5]
player_counts = list(range(3, 21))

# 結果を格納する辞書
total_used_nodes_per_choice = {c: [] for c in choices}

# 各選択肢について繰り返し
for c in choices:
    for player in player_counts:
        N = c * player
        QUBO = np.random.randn(N ** 2).reshape(N, N)

        # D-Waveサンプラーの設定
        dw_sampler = DWaveSampler(solver='Advantage_system4.1', token=token, endpoint=endpoint)
        sampler = EmbeddingComposite(dw_sampler)
        sampleset = sampler.sample_qubo(QUBO, num_reads=1)

        Qdict = {}
        for i in range(N):
            for j in range(N):
                if QUBO[i][j] != 0.0:
                    Qdict[(i, j)] = QUBO[i][j]

        adj = {}
        for k in Qdict.keys():
            adj[k] = 1
            
        embedding = find_embedding(adj, dw_sampler.edgelist)

        used_nodes = set(node for nodes in embedding.values() for node in nodes)
        total_used_nodes = len(used_nodes)

        # プレイヤー数に対して総使用ノード数を記録
        total_used_nodes_per_choice[c].append(total_used_nodes)

# 折れ線グラフの作成
plt.figure(figsize=(10, 6))
for c in choices:
    plt.plot(player_counts, total_used_nodes_per_choice[c], marker='o', label=f'Choice = {c}')

plt.title('Total Used Nodes by Player Count')
plt.xlabel('Player Count')
plt.ylabel('Total Used Nodes')
plt.xticks(player_counts)
plt.legend()
plt.grid()
file_name = f'Dwave_PNC_{min(choices)}_{max(choices)}_player_{min(player_counts)}_{max(player_counts)}.png'
plt.savefig(file_name)
plt.show()

